# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("hacker").getOrCreate()

In [3]:
data = spark.read.csv('./data/hack_data.csv', inferSchema=True, header=True)

In [5]:
data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [11]:
data.head(1)[0]

Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)

In [13]:
data.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [12]:
from pyspark.ml.feature import VectorAssembler

In [17]:
assembler = VectorAssembler(inputCols=[
'Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed'], outputCol='features')

In [21]:
final_data = assembler.transform(data)

In [22]:
final_data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)



In [23]:
from pyspark.ml.clustering import KMeans

In [24]:
kmeans = KMeans(featuresCol='features', k=3)

In [27]:
model = kmeans.fit(final_data)

In [31]:
results = model.transform(final_data)

In [29]:
centers = model.clusterCenters()

In [30]:
print(centers)

[array([4.09722222e+01, 8.87257847e+02, 5.20833333e-01, 7.13319444e+00,
       1.35347222e+01, 4.55445833e+01]), array([ 18.38461538, 254.87307692,   0.43956044,   3.09978022,
         7.9010989 ,  70.51747253]), array([ 24.74747475, 523.85191919,   0.56565657,   4.5159596 ,
         9.61616162,  62.39242424])]


In [35]:
results.select('prediction').show()

+----------+
|prediction|
+----------+
|         2|
|         0|
|         1|
|         1|
|         2|
|         2|
|         1|
|         1|
|         1|
|         2|
|         1|
|         1|
|         2|
|         1|
|         0|
|         1|
|         1|
|         1|
|         1|
|         2|
+----------+
only showing top 20 rows



In [38]:
results.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   91|
|         2|   99|
|         0|  144|
+----------+-----+



In [43]:
kmeans = KMeans(featuresCol='features', k=2)

In [44]:
model = kmeans.fit(final_data)

In [45]:
results = model.transform(final_data)

In [46]:
results.select('prediction').show()

+----------+
|prediction|
+----------+
|         1|
|         0|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         0|
|         1|
|         1|
|         1|
|         1|
|         1|
+----------+
only showing top 20 rows



In [47]:
results.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  170|
|         0|  164|
+----------+-----+



In [ ]:
# So just 2 hackers cause it was an even split 